In [1]:
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import BertConfig, BertForMaskedLM
from transformers import Trainer, TrainingArguments

from datasets import load_dataset
import hydra
from omegaconf import DictConfig

2023-11-17 16:46:34.442504: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-17 16:46:36.463033: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 16:46:38.332880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-17 16:46:53.514555: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
@hydra.main(version_base=None, config_path="../conf", config_name="config")
def main(cfg: DictConfig) -> None:
    return cfg

In [4]:
slice_data = load_dataset("csv", data_files="/home/so87pot/n0w0f/structllm/data/130k/slice.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
slice_data

DatasetDict({
    train: Dataset({
        features: ['slices'],
        num_rows: 137641
    })
})

In [16]:
slice_data['train'].column_names

['slices']

In [5]:
# Load the custom tokenizer using tokenizers library
self_tokenizer = Tokenizer.from_file("/home/so87pot/n0w0f/structllm/src/structllm/tokenizer/tokenizer-slice_130k.json")
self_wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=self_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

def tokenize_pad_and_truncate(texts):
        return self_wrapped_tokenizer(texts["slice"], truncation=True, padding="max_length", max_length=128)


In [11]:
train_dataset = load_dataset("csv", data_files="/home/so87pot/n0w0f/structllm/data/mb_1/train_matbench_perovskites_0.csv")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['slices', 'labels'],
        num_rows: 15142
    })
})

In [15]:
train_dataset = load_dataset("csv", data_files="/home/so87pot/n0w0f/structllm/data/mb_1/csv/train_matbench_mp_e_form_0.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['slices', 'labels'],
        num_rows: 106164
    })
})

In [8]:
tokenized_datasets = train_dataset.map(tokenize_pad_and_truncate, batched=True)

Map:   0%|          | 0/106163 [00:00<?, ? examples/s]

KeyError: 'slice'